In [1]:
import sys
sys.path.append('/Users/ik/Desktop/zepeto')
from a_python.rigging_class.rig_hier_maya_torch import *
import json
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import pickle as pkl
import scipy
import torch
import math
import time
import pytorch3d
from a_python.utils.rot import _rot_base

from pytorch3d.io import load_objs_as_meshes, save_obj
from pytorch3d.structures import Meshes
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVOrthographicCameras,
    OpenGLOrthographicCameras,
    FoVPerspectiveCameras, 
    PointLights, 
    RasterizationSettings, 
    MeshRenderer, 
    MeshRasterizer,  
    SoftPhongShader,
    SoftSilhouetteShader,
    SoftPhongShader
)
from pytorch3d.renderer.mesh import rasterize_meshes
from pytorch3d.renderer.mesh.rasterize_meshes import rasterize_meshes_python

from pytorch3d.renderer.blending import BlendParams
from pytorch3d.renderer.blending import sigmoid_alpha_blend, _sigmoid_alpha


# Setup
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

_vertices = copy.deepcopy(vertices)

bone :  lowerArm_L_scale num :  44
bone :  lowerArmTwist_L_scale num :  45
bone :  hand_L num :  61
bone :  lowerArmTwist_L num :  62


In [2]:
# 사진 index

idx_img = 11
idx_img = 8

# optimize parameters
n_iter = 50
alpha_loss_excede_mask = 10          # 마스크 넘어가는거에 얼마나 가중치 줄건지

visualize_pcd = False


In [3]:
def img_upperbody_maya_joints(keypts):
    '''maya관절 길이비율대로 상체 조인트 추가 : chest upper, chest, spine'''
    img_hip = (keypts[9,:2] + keypts[12,:2])/2
    img_neck = (keypts[2,:2] + keypts[5,:2])/2
    img_spine = (img_neck - img_hip) * len_joint_upper[-1] + img_hip
    img_chest = (img_neck - img_hip) * (len_joint_upper[-1] + len_joint_upper[-2]) + img_hip
    img_chsetupper = (img_neck - img_hip) * (len_joint_upper[-1] + len_joint_upper[-2] + len_joint_upper[-3])  + img_hip
    return img_hip, img_neck, img_spine, img_chest, img_chsetupper

def fitting_based_on_upperbody_joints(body, img_spine, img_hip, img_neck, _vertices):
    '''img_upperbody_maya_joints 함수 결과 사용해서 메쉬 스케일링으로 상체 조인트위치 맞춤'''
    # hip to spine 거리
    _len_spine_maya = np.linalg.norm(body.head - body.childs[0].head) 
    _len_spine_img = np.linalg.norm(img_spine - img_hip)

    # 상체 조인트 맞춤
    _spine_trans = -(_len_spine_maya - _len_spine_img) * \
        ((-body.head + body.childs[0].head) / np.linalg.norm(body.head - body.childs[0].head))
    body.childs[0].trans_head(_spine_trans, _vertices)
    _scale_spine_hier = (np.linalg.norm(img_hip - img_neck) * len_joint_upper[-1]) / \
        (np.linalg.norm(body.childs[0].head - body.childs[0].childs[0].head))
    body.childs[0].scale_iso_hier(_scale_spine_hier, _vertices)

def _rot_base(axis, angle):
    zero = torch.zeros_like(angle)
    one = torch.ones_like(angle)
    if axis == 'x':
        return torch.tensor(
            [
                torch.stack([one, zero, zero]),
                torch.stack([0, torch.cos(angle), -torch.sin(angle)]),
                torch.stack([0, torch.sin(angle), torch.cos(angle)])
            ]
        )

    if axis == 'y':
        return torch.tensor(
            [
                torch.stack([torch.cos(angle), zero, torch.sin(angle)]),
                torch.stack([zero, one, zero]),
                torch.stack([-torch.sin(angle), zero, torch.cos(angle)])
            ]
        )

    if axis == 'z':
        return torch.stack(
            [
                torch.stack([torch.cos(angle), -torch.sin(angle), zero]),
                torch.stack([torch.sin(angle), torch.cos(angle), zero]),
                torch.stack([zero, zero, one]),
            ]
        )


In [4]:
folder = idx_img
path_img = "/Users/ik/Downloads/test_set_new/{}/test_resized.png".format(folder)
path_json = "/Users/ik/Downloads/test_set_new/{}/test_resized_keypoints.json".format(folder)
path_dp = "/Users/ik/Downloads/test_set_new/{}/dp_dump.pkl".format(folder)

with open(path_json, 'r') as f:
    data = json.load(f)
with open(path_dp, 'rb') as f:
    [img_seg, img_v, img_u,_] = pkl.load(f)
h,w = img_seg.shape
keypts = torch.tensor(data['people'][0]['pose_keypoints_2d']).reshape([-1,3])
img = cv.imread(path_img)

%matplotlib widget



In [5]:
_vertices = copy.deepcopy(vertices)
body = rig_class(65)
print(body.childs[0].childs[0].childs[0].childs[1].childs[0].childs[0].childs[0].name)
chestUpper_scale = body.childs[0].childs[0].childs[0]
shoulder_L_scale = body.childs[0].childs[0].childs[0].childs[1]
lowerarm_scale = body.childs[0].childs[0].childs[0].childs[1].childs[0].childs[0].childs[0]
lowerarm_twist_scale = body.childs[0].childs[0].childs[0].childs[1].childs[0].childs[0].childs[0].childs[0]

# chestUpper_scale.scale_iso_hier(5, _vertices)
# shoulder_L_scale.scale_iso_hier(1.5, _vertices)
# lowerarm_scale.scale_iso_hier(5, _vertices)
# lowerarm_scale.rot(_rot_base('z', torch.tensor(math.pi/3)), _vertices, inplace=False)
# lowerarm_twist_scale.scale_iso_hier(5, _vertices)
shoulder_L_scale.rot(_rot_base('z', torch.tensor(math.pi/3)), _vertices, inplace=False)
# lowerarm_twist_scale.rot(_rot_base('z', torch.tensor(math.pi/3)), _vertices, inplace=False)

lowerArm_L_scale


In [ ]:
lowerarm_twist_scale.weight_vertices_for_bone
lowerarm_twist_scale.idx_vertices_for_bone

_weight_sum = torch.zeros([vertices.shape[0]])
tmp = body.concat_weights(_weight_sum)
print(_weight_sum[8665])
print(_weight_sum[8657])
print(_weight_sum[8633])
print(_weight_sum[8624])
print(_weight_sum[7360])
print(_weight_sum[7420])
print(_weight_sum[7647])

f_index = np.arange(_vertices.shape[0])[_weight_sum <0.8]
print(f_index)
print(_weight_sum[7432])

In [7]:
_list_joint = []
pcd = o3d.geometry.PointCloud()

_vertices_opt = copy.deepcopy(_vertices)
body_copy = copy.deepcopy(body)


_vertices_draw = np.asarray(copy.deepcopy(_vertices_opt.detach()))

pcd.points = o3d.utility.Vector3dVector(_vertices_draw)

_color = np.zeros([9067,3])*np.array([0.5,0.3,1])

# _color[_weight_sum<=0.8,:] = np.array([0,1,0])

pcd.colors = o3d.utility.Vector3dVector(_color)

_list_joint.append(pcd)
body.draw_joint(_list_joint)
o3d.visualization.draw_geometries(_list_joint)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


2022-11-26 02:02:43.396 Python[83125:14787965] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


: 

# Visualize

In [ ]:
_list_joint = []
pcd = o3d.geometry.PointCloud()

_vertices_opt = copy.deepcopy(_vertices)
body_copy = copy.deepcopy(body)


_vertices_draw = np.asarray(copy.deepcopy(_vertices_opt.detach()))

# _vertices_draw = np.asarray(copy.deepcopy(_vertices.detach()))
pcd.points = o3d.utility.Vector3dVector(_vertices_draw)
_color = np.ones([9067,3])*np.array([0.5,0.3,1])
pcd.colors = o3d.utility.Vector3dVector(_color)
# _pcd = o3d.geometry.PointCloud()
# _pcd.points = o3d.utility.Vector3dVector(vertices)
# _list_joint.append(_pcd)
_list_joint.append(pcd)
body.draw_joint(_list_joint)
o3d.visualization.draw_geometries(_list_joint)